In [1]:
import pickle
import pandas as pd
import numpy as np
from API import K
import requests
import json
import googlemaps

In [2]:
#import dataframe and drop nans
df_2021 = pd.read_csv('data/2021/2021_combined.csv')
#drop nans
df_2021 = df_2021.loc[df_2021.SK_CODE.isnull() != True]

In [113]:
df_2021

,Unnamed: 0,Jaarmaand,Jaar,Maand,Weeknr,Date,UNLO_herkomst,UNLO_bestemming,v15_1_Scheepstype_RWS,SK_CODE,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw,nst2007_nw
0,0,2101,2021,1,53,2021-01-01 00:00:00+00:00,NLAMS,BEANR,2,M12,8000.0,3,0.0,0,3.0,7.4
1,1,2101,2021,1,53,2021-01-01 00:00:00+00:00,DEGDO,BEANR,2,M8,2860.0,3,0.0,0,3.0,7.4
2,2,2101,2021,1,53,2021-01-01 00:00:00+00:00,NLVLI,BEWND,2,M9,3520.0,7,3000000.0,0,3.0,7.2
3,3,2101,2021,1,53,2021-01-01 00:00:00+00:00,DEHBO,NLRTM,2,M8,1740.0,7,500000.0,0,8.0,8.2
4,4,2101,2021,1,53,2021-01-01 00:00:00+00:00,NLAMS,DEDUI,16,BIIa-1,2510.0,7,2004000.0,0,3.0,7.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385567,385567,2112,2021,12,52,2021-12-31 22:00:00+00:00,DEDMG,BEGNE,16,BIIa-1,3550.0,3,0.0,0,3.0,7.4
385568,385568,2112,2021,12,52,2021-12-31 22:00:00+00:00,NLRTM,BEGNE,2,M10,3940.0,7,3392000.0,0,3.0,7.2
385569,385569,2112,2021,12,52,2021-12-31 22:00:00+00:00,DEMHG,BEANR,2,M9,3510.0,3,0.0,0,3.0,7.2
385570,385570,2112,2021,12,52,2021-12-31 22:00:00+00:00,NLRTM,BEGNE,2,M9,4290.0,7,2000000.0,0,8.0,8.5


In [3]:
# group trips by origin, destination and category
df_t = df_2021.groupby(by=['UNLO_herkomst', 'UNLO_bestemming', 'SK_CODE']).count().reset_index().sort_values(
    by='Jaarmaand', ascending=False).iloc[:, :4].reset_index(drop=True)

In [4]:
# give columns english names
df_t.rename(
    columns={'UNLO_herkomst': 'origin', 'UNLO_bestemming': 'destination', 'SK_CODE': 'type', 'Unnamed: 0': 'trips'},
    inplace=True)

In [5]:
# subset inland trips
df_t = df_t.loc[(df_t.origin.str.contains('NL')) & (df_t.destination.str.contains('NL'))]
df_t.reset_index(inplace=True, drop=True)

In [6]:
#create dict to fill with total number of trips on path for each ship type
df_t_dict = {}
# now fill this dict
# loop over data frame
for i in range(len(df_t)):
    #define temp variables for ease
    origin = df_t['origin'][i]
    destination = df_t['destination'][i]
    ship_type = df_t['type'][i]
    trip_count = df_t['trips'][i]
    #only make new column if trip and its inverse are not in dict yet. Otherwise append.
    if (origin, destination, ship_type) in df_t_dict.keys():
        df_t_dict[(origin, destination, ship_type)] += trip_count
    elif (destination, origin, ship_type) in df_t_dict.keys():
        df_t_dict[(destination, origin, ship_type)] += trip_count
    else:
        df_t_dict[(origin, destination, ship_type)] = trip_count
df_t_dict

{('NLAMS', 'NLRTM', 'M8'): 3510,
 ('NLAMS', 'NLRTM', 'M12'): 2076,
 ('NLRTM', 'NLMOE', 'M8'): 1584,
 ('NLRTM', 'NLRTM', 'M8'): 715,
 ('NLAMS', 'NLRTM', 'M9'): 1316,
 ('NLRTM', 'NLWLK', 'M3'): 973,
 ('NLOOS', 'NLTLB', 'M8'): 688,
 ('NLRTM', 'NLAMS', 'M6'): 881,
 ('NLAER', 'NLAMS', 'BI'): 824,
 ('NLDHR', 'NLDHR', 'M0'): 411,
 ('NLTLB', 'NLRTM', 'M8'): 623,
 ('NLRTM', 'NLOOS', 'M8'): 727,
 ('NLHAR', 'NLWTE', 'M12'): 690,
 ('NLHGL', 'NLRTM', 'M8'): 652,
 ('NLRTM', 'NLVLI', 'M8'): 658,
 ('NLRTM', 'NLRTM', 'M12'): 363,
 ('NLMOE', 'NLOOS', 'M8'): 615,
 ('NLVEG', 'NLRTM', 'M6'): 575,
 ('NLAMS', 'NLRTM', 'M11'): 666,
 ('NLMOE', 'NLAPN', 'M8'): 604,
 ('NLVEG', 'NLRTM', 'M7'): 581,
 ('NLAMS', 'NLMOE', 'M8'): 588,
 ('NLRTM', 'NLMEP', 'M8'): 568,
 ('NLSTD', 'NLSTD', 'M0'): 282,
 ('NLAMS', 'NLLMR', 'BIIL-1'): 553,
 ('NLNIJ', 'NLWAS', 'M8'): 508,
 ('NLWSP', 'NLAER', 'M6'): 510,
 ('NLDOR', 'NLRTM', 'M8'): 459,
 ('NLAMS', 'NLHAS', 'M8'): 436,
 ('NLRTM', 'NLDZL', 'M8'): 415,
 ('NLRTM', 'NLTIE', 'M8'): 4

## Compute vessel sum per route
Idea: make similar dict as above first, but then with total amount of trips per route. Later on: add column for each ship.
This will result in a column for each ship type and an entry with the amount of ships of that type that have travelled.

In [7]:
# now only sort by origin and destination
all_trips_ranked = df_2021.groupby(by=['UNLO_herkomst', 'UNLO_bestemming']).count().reset_index().sort_values(
    by='Jaarmaand', ascending=False).iloc[:, :3].reset_index(drop=True)

#again, rename columns
all_trips_ranked.rename(
    columns={'UNLO_herkomst': 'origin', 'UNLO_bestemming': 'destination', 'SK_CODE': 'type', 'Unnamed: 0': 'trips'},
    inplace=True)

#remove non nl
all_trips_ranked = all_trips_ranked.loc[(all_trips_ranked.origin.str.contains('NL')) & (all_trips_ranked.destination.str.contains('NL'))]

#reset index to be able to loop over all values later on
all_trips_ranked.reset_index(inplace=True, drop=True)

# make dict to store trips
trip_dict_main = {}

In [8]:
# now fill this dict, same procedure as above
for i in range(len(all_trips_ranked)):
    origin = all_trips_ranked['origin'][i]
    destination = all_trips_ranked['destination'][i]
    trip_count = all_trips_ranked['trips'][i]

    if (origin, destination) in trip_dict_main.keys():
        trip_dict_main[(origin, destination)] += trip_count
    elif (destination, origin) in trip_dict_main.keys():
        trip_dict_main[(destination, origin)] += trip_count
    else:
        trip_dict_main[(origin, destination)] = trip_count

In [9]:
# now sort values
trip_dict_main = dict(sorted(trip_dict_main.items(), key=lambda item: item[1], reverse=True))

# make dataframe dict wih a key for each column
trip_dict_df_main = {'origin': [], 'destination': [], 'trip_count': []}
for i in df_2021.SK_CODE.unique():
    trip_dict_df_main[i] = []

# now fill this dict
for key, value in trip_dict_main.items():
    # append values from dict a
    trip_dict_df_main['origin'].append(key[0])
    trip_dict_df_main['destination'].append(key[1])
    trip_dict_df_main['trip_count'].append(value)
    #now append values from other dict, for each ship type
    for ship_type in df_2021.SK_CODE.unique():
        #if ships have travelled from A to B, append this value
        if (key[0], key[1], ship_type) in df_t_dict.keys():
            trip_dict_df_main[ship_type].append(df_t_dict[(key[0], key[1], ship_type)])
        else:
            try:
                # else, maybe trips on path A to B are stored as B to A
                trip_dict_df_main[ship_type].append(df_t_dict[(key[1], key[0], ship_type)])
            except:
                # no? Then no ships of this type have travelled this path, append 0
                trip_dict_df_main[ship_type].append(0)

In [10]:
df_main = pd.DataFrame.from_dict(trip_dict_df_main)

In [11]:
df_main

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,2076,3510,1316,102,0,427,0,...,5,7,11,5,39,4,0,0,0,0
1,NLRTM,NLMOE,2686,70,1584,144,2,0,56,0,...,2,0,1,0,1,0,0,0,0,0
2,NLRTM,NLVLI,2153,190,658,414,10,0,140,0,...,0,2,2,1,0,0,0,1,0,0
3,NLAMS,NLAER,2119,6,132,0,2,0,7,0,...,0,1,5,37,0,3,0,1,1,0
4,NLRTM,NLRTM,1794,363,715,92,15,0,164,1,...,4,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10588,NLBZM,NLVSN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10589,NLBZM,NLVER,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10590,NLBZM,NLSMB,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10591,NLBZM,NLSHH,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#remove entries with same origin and destination
df_main = df_main.loc[df_main.origin!=df_main.destination]

In [4]:
df_main

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,2076,3510,1316,102,0,427,0,...,5,7,11,5,39,4,0,0,0,0
1,NLRTM,NLMOE,2686,70,1584,144,2,0,56,0,...,2,0,1,0,1,0,0,0,0,0
2,NLRTM,NLVLI,2153,190,658,414,10,0,140,0,...,0,2,2,1,0,0,0,1,0,0
3,NLAMS,NLAER,2119,6,132,0,2,0,7,0,...,0,1,5,37,0,3,0,1,1,0
5,NLVEG,NLRTM,1484,0,1,0,0,0,0,0,...,0,0,17,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10588,NLBZM,NLVSN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10589,NLBZM,NLVER,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10590,NLBZM,NLSMB,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10591,NLBZM,NLSHH,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#save dataframe
# pickle.dump( df_main, open( "data/df_trips_per_path.p", "wb" ) )
df_main = pickle.load( open( "data/df_trips_per_path.p", "rb" ) )

## Generate random data

In [12]:
df_chance = df_main.copy()

In [13]:
df_chance

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,2076,3510,1316,102,0,427,0,...,5,7,11,5,39,4,0,0,0,0
1,NLRTM,NLMOE,2686,70,1584,144,2,0,56,0,...,2,0,1,0,1,0,0,0,0,0
2,NLRTM,NLVLI,2153,190,658,414,10,0,140,0,...,0,2,2,1,0,0,0,1,0,0
3,NLAMS,NLAER,2119,6,132,0,2,0,7,0,...,0,1,5,37,0,3,0,1,1,0
4,NLRTM,NLRTM,1794,363,715,92,15,0,164,1,...,4,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10588,NLBZM,NLVSN,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10589,NLBZM,NLVER,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10590,NLBZM,NLSMB,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10591,NLBZM,NLSHH,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
type_list = list(df_main.columns[3:])

In [15]:
for i in type_list:
    df_chance[i] = df_chance[i]/df_chance.trip_count

In [20]:
# pickle.dump( df_chance, open( "data/probability_df.p", "wb" ) )
df_chance = pickle.load(  open( "data/probability_df.p", "rb" ) )

In [21]:
df_chance

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,0.208790,0.353012,0.132354,0.010258,0.0,0.042945,0.000000,...,0.000503,0.000704,0.001106,0.000503,0.003922,0.000402,0.0,0.000000,0.000000,0.0
1,NLRTM,NLMOE,2686,0.026061,0.589724,0.053611,0.000745,0.0,0.020849,0.000000,...,0.000745,0.000000,0.000372,0.000000,0.000372,0.000000,0.0,0.000000,0.000000,0.0
2,NLRTM,NLVLI,2153,0.088249,0.305620,0.192290,0.004645,0.0,0.065026,0.000000,...,0.000000,0.000929,0.000929,0.000464,0.000000,0.000000,0.0,0.000464,0.000000,0.0
3,NLAMS,NLAER,2119,0.002832,0.062294,0.000000,0.000944,0.0,0.003303,0.000000,...,0.000000,0.000472,0.002360,0.017461,0.000000,0.001416,0.0,0.000472,0.000472,0.0
4,NLRTM,NLRTM,1794,0.202341,0.398551,0.051282,0.008361,0.0,0.091416,0.000557,...,0.002230,0.000000,0.000000,0.000000,0.001115,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10588,NLBZM,NLVSN,1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
10589,NLBZM,NLVER,1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
10590,NLBZM,NLSMB,1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
10591,NLBZM,NLSHH,1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [22]:
test_100 = df_chance.head(100)

In [16]:
test_100

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,0.208790,0.353012,0.132354,0.010258,0.0,0.042945,0.000000,...,0.000503,0.000704,0.001106,0.000503,0.003922,0.000402,0.0,0.000000,0.000000,0.0
1,NLRTM,NLMOE,2686,0.026061,0.589724,0.053611,0.000745,0.0,0.020849,0.000000,...,0.000745,0.000000,0.000372,0.000000,0.000372,0.000000,0.0,0.000000,0.000000,0.0
2,NLRTM,NLVLI,2153,0.088249,0.305620,0.192290,0.004645,0.0,0.065026,0.000000,...,0.000000,0.000929,0.000929,0.000464,0.000000,0.000000,0.0,0.000464,0.000000,0.0
3,NLAMS,NLAER,2119,0.002832,0.062294,0.000000,0.000944,0.0,0.003303,0.000000,...,0.000000,0.000472,0.002360,0.017461,0.000000,0.001416,0.0,0.000472,0.000472,0.0
4,NLRTM,NLRTM,1794,0.202341,0.398551,0.051282,0.008361,0.0,0.091416,0.000557,...,0.002230,0.000000,0.000000,0.000000,0.001115,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NLNIJ,NLAMS,352,0.011364,0.411932,0.028409,0.065341,0.0,0.000000,0.000000,...,0.000000,0.011364,0.000000,0.000000,0.002841,0.000000,0.0,0.000000,0.002841,0.0
96,NLFAR,NLRTM,342,0.000000,0.903509,0.029240,0.005848,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.0,0.000000,0.000000,0.0
97,NLRTM,NLVSN,342,0.000000,0.602339,0.368421,0.000000,0.0,0.011696,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
98,NLPER,NLAMS,328,0.027439,0.387195,0.097561,0.000000,0.0,0.018293,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [62]:
def random_vessel_generator(df_prob):
    # create dict to store random prob based values later on
    main_dict = {i:[] for i in df_prob.columns}

    # loop over all rows of the probability dataframe
    for i in range(len(df_prob)):
        # copy origin, destination and count from original df
        for x in df_prob.columns[:3]:
            main_dict[x].append(df_prob.iloc[i,:][x])

        # find probability, items to pick, and the number of vessels to generate in total
        prob = list(df_prob.iloc[i, 3:].values)
        to_pick = list(df_prob.columns)
        to_pick = to_pick[3:]
        count = df_prob['trip_count'][i]

        # generate random vessels
        rand_vessels = np.random.choice(a=to_pick, size=count, replace=True, p=prob)
        unique, counts = np.unique(rand_vessels, return_counts=True)
        temp_dict = dict(zip(unique,counts))

        # append amount of random generated vessels right dict list
        for key in list(main_dict.keys())[3:]:
            if key in temp_dict.keys():
                main_dict[key].append(temp_dict[key])
            else:
                main_dict[key].append(0)

        # now make dict
        df_return = pd.DataFrame.from_dict(main_dict)

    return df_return

In [65]:
import time
%time vs = random_vessel_generator(test_100)

CPU times: total: 312 ms
Wall time: 305 ms


## Now translate random vessels fuel consumption value for each path
Idea: take 1 vessel as a basis, and assume that fuel consumption is linearly correlated with average engine power, to be able to use index
Capacity is likely x vessels per day of a certain type for a charging unit, depending on the charging time.
Hence, computing the average amount of that type of vehicles that comes by everyday makes sense.

In [72]:
# generate a random set to calculate path values with
df_gen_vessels = random_vessel_generator(test_100)

In [69]:
ship_data = pd.read_excel('data/ship_types.xlsx')
ship_data.fillna(0, inplace=True)

In [70]:
ship_data.loc[:,['RWS-class', 'Factor']]

,RWS-class,Factor
0,M0,0.000000
1,M1,1.000000
2,M2,1.714286
3,M3,2.485714
4,M4,2.485714
5,M5,2.485714
6,M6,3.942857
7,M7,3.942857
8,M8,8.142857
9,M9,8.142857


In [73]:
df_gen_vessels

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,2159,3431,1336,122,0,418,0,...,7,3,14,4,37,7,0,0,0,0
1,NLRTM,NLMOE,2686,63,1627,110,1,0,46,0,...,0,0,2,0,0,0,0,0,0,0
2,NLRTM,NLVLI,2153,181,673,421,9,0,144,0,...,0,1,2,1,0,0,0,3,0,0
3,NLAMS,NLAER,2119,5,128,0,1,0,10,0,...,0,1,4,35,0,4,0,0,1,0
4,NLRTM,NLRTM,1794,364,725,93,10,0,171,1,...,4,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NLNIJ,NLAMS,352,3,154,8,24,0,0,0,...,0,2,0,0,1,0,0,0,1,0
96,NLFAR,NLRTM,342,0,303,13,1,0,0,0,...,0,0,0,0,5,0,0,0,0,0
97,NLRTM,NLVSN,342,0,207,120,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
98,NLPER,NLAMS,328,6,126,35,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
ship_data = dict(zip(ship_data['RWS-class'],ship_data['Factor']))

In [75]:
def flow_computation(df):
    # create dict to store path based values
    flows = {}
    # loop over data frame
    for i in range(len(df)-1):
        # subset all data ship type data
        a = df.iloc[:,3:]
        # flow is initially 0
        flow = 0
        #add number of ships times specific ship type weighing factor
        for row in a.columns:
            flow += ship_data[row] * a[row][i]
        # store flow, divide by 365 to get daily flow
        flows[(df.origin[i],df.destination[i])]= (flow/365)
    return flows

In [76]:
flows = flow_computation(df_gen_vessels)

In [77]:
flows

{('NLRTM', 'NLAMS'): 231.86943248532296,
 ('NLRTM', 'NLMOE'): 59.17072407045009,
 ('NLRTM', 'NLVLI'): 46.508493150684934,
 ('NLAMS', 'NLAER'): 21.328454011741684,
 ('NLRTM', 'NLRTM'): 41.673424657534234,
 ('NLVEG', 'NLRTM'): 15.091193737769082,
 ('NLTLB', 'NLRTM'): 18.00908023483366,
 ('NLLEY', 'NLAMS'): 13.309823874755383,
 ('NLOSS', 'NLRTM'): 14.652759295499019,
 ('NLZWO', 'NLAMS'): 15.07123287671233,
 ('NLUTC', 'NLRTM'): 20.881722113502935,
 ('NLRTM', 'NLKAM'): 21.554285714285708,
 ('NLHGL', 'NLRTM'): 18.096046966731897,
 ('NLLMR', 'NLAMS'): 22.848454011741687,
 ('NLRTM', 'NLWLK'): 6.717416829745598,
 ('NLAMS', 'NLMOE'): 18.417377690802347,
 ('NLAER', 'NLRTM'): 17.400469667318983,
 ('NLRTM', 'NLAPN'): 11.896046966731898,
 ('NLTNZ', 'NLVLI'): 17.44954990215264,
 ('NLDOR', 'NLRTM'): 18.832876712328765,
 ('NLAMS', 'NLDEV'): 6.741996086105675,
 ('NLGOR', 'NLRTM'): 17.858864970645794,
 ('NLRTM', 'NLNIJ'): 15.337299412915849,
 ('NLBZM', 'NLRTM'): 13.04289628180039,
 ('NLWSP', 'NLAER'): 10

In [78]:
#sort flows from large to small
flows = dict(sorted(flows.items(), key=lambda item: item[1], reverse=True))

In [79]:
flows

{('NLRTM', 'NLAMS'): 231.86943248532296,
 ('NLRTM', 'NLMOE'): 59.17072407045009,
 ('NLRTM', 'NLVLI'): 46.508493150684934,
 ('NLRTM', 'NLRTM'): 41.673424657534234,
 ('NLLMR', 'NLAMS'): 22.848454011741687,
 ('NLHAR', 'NLWTE'): 21.89135029354207,
 ('NLRTM', 'NLKAM'): 21.554285714285708,
 ('NLAMS', 'NLAER'): 21.328454011741684,
 ('NLUTC', 'NLRTM'): 20.881722113502935,
 ('NLVLI', 'NLAMS'): 19.795068493150684,
 ('NLDOR', 'NLRTM'): 18.832876712328765,
 ('NLAMS', 'NLMOE'): 18.417377690802347,
 ('NLHGL', 'NLRTM'): 18.096046966731897,
 ('NLTLB', 'NLRTM'): 18.00908023483366,
 ('NLGOR', 'NLRTM'): 17.858864970645794,
 ('NLTNZ', 'NLVLI'): 17.44954990215264,
 ('NLAER', 'NLRTM'): 17.400469667318983,
 ('NLRTM', 'NLOOS'): 17.04430528375734,
 ('NLRTM', 'NLTNZ'): 16.324931506849314,
 ('NLNIJ', 'NLWAS'): 15.976203522504893,
 ('NLOOS', 'NLTLB'): 15.604383561643832,
 ('NLRTM', 'NLNIJ'): 15.337299412915849,
 ('NLVEG', 'NLRTM'): 15.091193737769082,
 ('NLZWO', 'NLAMS'): 15.07123287671233,
 ('NLOSS', 'NLRTM'): 1

## Get and clean harbour data NL, Ger and Bel

In [6]:
# now get coordinates harbours, in three files, check encoding frist below
pd.read_csv('data/harbour_codes_coords/2021-2 UNLOCODE CodeListPart1.csv', encoding="ISO-8859-1", header=None, sep=",",
            index_col=None)

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,AD,NaN,.ANDORRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AD,ALV,Andorra la Vella,Andorra la Vella,NaN,--34-6--,AI,601.0,NaN,4230N 00131E,NaN
2,NaN,AD,CAN,Canillo,Canillo,NaN,--3-----,RL,307.0,NaN,4234N 00135E,NaN
3,NaN,AD,ENC,Encamp,Encamp,NaN,--3-----,RL,307.0,NaN,4232N 00134E,NaN
4,NaN,AD,ESC,Escaldes-Engordany,Escaldes-Engordany,NaN,--3-----,RL,307.0,NaN,4231N 00133E,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
54611,NaN,FR,ZTG,Zetting,Zetting,57,--3-----,RL,1901.0,NaN,4905N 00708E,NaN
54612,NaN,FR,ZL2,Zilia,Zilia,2B,-----6--,RL,1901.0,NaN,4231N 00854E,NaN
54613,NaN,FR,ZWL,Zinswiller,Zinswiller,67,--3-----,RL,1901.0,NaN,4855N 00735E,NaN
54614,NaN,FR,PCZ,Zoteux,Zoteux,62,-----6--,RL,1001.0,NaN,5037N 00153E,NaN


In [7]:
# now identify all files to combine
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk('data/harbour_codes_coords'):
    f.extend(filenames)
    break

In [8]:

#empty list to store data
li = []

#read everything in as object for now
for filename in f:
    df = pd.read_csv("data/harbour_codes_coords/" + str(filename), sep=',', index_col=None, header=None,
                     encoding="ISO-8859-1")
    print(filename, 'handled')
    li.append(df)

2021-2 UNLOCODE CodeListPart1.csv handled
2021-2 UNLOCODE CodeListPart2.csv handled
2021-2 UNLOCODE CodeListPart3.csv handled


In [9]:
#create df
df_harbour_codes = pd.concat(li, axis=0, ignore_index=True)
df_benelux=df_harbour_codes.copy()
#subset Dutch harbours
# df_benelux = df_harbour_codes.loc[(df_harbour_codes[1] == 'NL')]

# uncomment below to loc benelux, for now only use NL because of scope
# df_benelux = df_harbour_codes.loc[
#     (df_harbour_codes[1] == 'NL') | (df_harbour_codes[1] == 'DE') | (df_harbour_codes[1] == 'BE') | (
#                 df_harbour_codes[1] == 'LU')]

In [10]:
#fillna
df_benelux.fillna(np.nan, inplace=True)

In [11]:
# give appropriate column names
df_benelux.rename(columns={0: 'mutation', 1: 'country', 2: 'city_abbr', 3: 'city_full', 4: 'city_full_2', 5: 'subdivision', 6: 'function', 7:'status', 8:'date', 9:'iata_code', 10: 'coords', 11: 'comments'}, inplace=True)

In [12]:
df_benelux

,mutation,country,city_abbr,city_full,city_full_2,subdivision,function,status,date,iata_code,coords,comments
0,NaN,AD,NaN,.ANDORRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AD,ALV,Andorra la Vella,Andorra la Vella,NaN,--34-6--,AI,601.0,NaN,4230N 00131E,NaN
2,NaN,AD,CAN,Canillo,Canillo,NaN,--3-----,RL,307.0,NaN,4234N 00135E,NaN
3,NaN,AD,ENC,Encamp,Encamp,NaN,--3-----,RL,307.0,NaN,4232N 00134E,NaN
4,NaN,AD,ESC,Escaldes-Engordany,Escaldes-Engordany,NaN,--3-----,RL,307.0,NaN,4231N 00133E,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
115984,NaN,ZW,STH,Southerton,Southerton,NaN,--3-----,RL,201.0,NaN,1751S 03101E,NaN
115985,NaN,ZW,THJ,Thompson Junction,Thompson Junction,NaN,-23-----,RL,701.0,NaN,1800S 02626E,NaN
115986,NaN,ZW,VFA,Victoria Falls,Victoria Falls,NaN,---4----,AI,9501.0,NaN,NaN,NaN
115987,NaN,ZW,ZMZ,Zimbabwe,Zimbabwe,MV,1-3-----,RL,1401.0,NaN,2016S 03055E,NaN


In [13]:
# subset relevant columns
# df_benelux = df_benelux.loc[:,['country','city_abbr','function', 'coords']]

In [14]:
#reset index
df_benelux.reset_index(inplace=True,drop=True)

In [15]:
# drop rows with nan values for necessary columns
# df_benelux.dropna(subset=['country', 'city_abbr', 'coords'], inplace = True)

In [16]:
# # convert columns to be normal strings
# for i in df_benelux.columns:
#     df_benelux[i] = df_benelux[i].astype('|S80')
#     df_benelux[i] = df_benelux[i].apply(lambda x: x.decode('utf-8'))

In [17]:
df_benelux.replace('nan', np.nan)

,mutation,country,city_abbr,city_full,city_full_2,subdivision,function,status,date,iata_code,coords,comments
0,NaN,AD,NaN,.ANDORRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AD,ALV,Andorra la Vella,Andorra la Vella,NaN,--34-6--,AI,601.0,NaN,4230N 00131E,NaN
2,NaN,AD,CAN,Canillo,Canillo,NaN,--3-----,RL,307.0,NaN,4234N 00135E,NaN
3,NaN,AD,ENC,Encamp,Encamp,NaN,--3-----,RL,307.0,NaN,4232N 00134E,NaN
4,NaN,AD,ESC,Escaldes-Engordany,Escaldes-Engordany,NaN,--3-----,RL,307.0,NaN,4231N 00133E,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
115984,NaN,ZW,STH,Southerton,Southerton,NaN,--3-----,RL,201.0,NaN,1751S 03101E,NaN
115985,NaN,ZW,THJ,Thompson Junction,Thompson Junction,NaN,-23-----,RL,701.0,NaN,1800S 02626E,NaN
115986,NaN,ZW,VFA,Victoria Falls,Victoria Falls,NaN,---4----,AI,9501.0,NaN,NaN,NaN
115987,NaN,ZW,ZMZ,Zimbabwe,Zimbabwe,MV,1-3-----,RL,1401.0,NaN,2016S 03055E,NaN


In [18]:
df_benelux['harbour_code'] = df_benelux.country+df_benelux.city_abbr

In [19]:
df_benelux = df_benelux.loc[:,['country','city_abbr','city_full','function','coords','harbour_code']]

In [20]:
# save cleaned data
df_benelux.to_csv('data/cleaned_harbours.csv')

## Final step: assign harbours to nodes

In [62]:
test_100

,origin,destination,trip_count,M12,M8,M9,BIIa-1,BII-6b,M10,BII-6l,...,C4,C2l,M0,B04,BII-2L,B02,C1b,C2b,B01,C1l
0,NLRTM,NLAMS,9943,0.208790,0.353012,0.132354,0.010258,0.0,0.042945,0.000000,...,0.000503,0.000704,0.001106,0.000503,0.003922,0.000402,0.0,0.000000,0.000000,0.0
1,NLRTM,NLMOE,2686,0.026061,0.589724,0.053611,0.000745,0.0,0.020849,0.000000,...,0.000745,0.000000,0.000372,0.000000,0.000372,0.000000,0.0,0.000000,0.000000,0.0
2,NLRTM,NLVLI,2153,0.088249,0.305620,0.192290,0.004645,0.0,0.065026,0.000000,...,0.000000,0.000929,0.000929,0.000464,0.000000,0.000000,0.0,0.000464,0.000000,0.0
3,NLAMS,NLAER,2119,0.002832,0.062294,0.000000,0.000944,0.0,0.003303,0.000000,...,0.000000,0.000472,0.002360,0.017461,0.000000,0.001416,0.0,0.000472,0.000472,0.0
4,NLRTM,NLRTM,1794,0.202341,0.398551,0.051282,0.008361,0.0,0.091416,0.000557,...,0.002230,0.000000,0.000000,0.000000,0.001115,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NLNIJ,NLAMS,352,0.011364,0.411932,0.028409,0.065341,0.0,0.000000,0.000000,...,0.000000,0.011364,0.000000,0.000000,0.002841,0.000000,0.0,0.000000,0.002841,0.0
96,NLFAR,NLRTM,342,0.000000,0.903509,0.029240,0.005848,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.0,0.000000,0.000000,0.0
97,NLRTM,NLVSN,342,0.000000,0.602339,0.368421,0.000000,0.0,0.011696,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
98,NLPER,NLAMS,328,0.027439,0.387195,0.097561,0.000000,0.0,0.018293,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [105]:
h_list = list(set(list(test_100.origin.unique()))|set(list(test_100.destination.unique())))

In [106]:
import re
h_list = [re.sub("NL", "", i) for i in h_list]

In [107]:
df_h_nl = df_benelux.loc[(df_benelux.city_abbr.isin(h_list))&(df_benelux.country == 'NL')]
df_h_nl.reset_index(inplace=True, drop=True)

In [108]:
df_h_nl['lat']=0
df_h_nl['lon']=0

C:\Users\Fabia\AppData\Local\Temp\ipykernel_26176\4140986558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h_nl['lat']=0
C:\Users\Fabia\AppData\Local\Temp\ipykernel_26176\4140986558.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h_nl['lon']=0


In [35]:
for i,city in enumerate(df_h_nl.city_full):
    r = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={'Haven van',str(city)}&key={K}")
    results = json.loads(r.content)
    lat_r = results['results'][0]['geometry']['location']['lat']
    lon_r = results['results'][0]['geometry']['location']['lng']
    df_h_nl['lat'][i] = lat_r
    df_h_nl['lon'][i] = lon_r


In [92]:
df_h_nl

,mutation,country,city_abbr,city_full,city_full_2,subdivision,function,status,date,iata_code,coords,comments,harbour_code
78676,0,NL,ABL,Alblasserdam,Alblasserdam,0,1-------,AF,9602.0,0,0,0,NLABL
78678,0,NL,ALK,Alkmaar,Alkmaar,NH,123-----,AF,1601.0,0,5238N 00445E,0,NLALK
78680,0,NL,ALM,Almelo,Almelo,OV,123-----,AF,1601.0,QYL,5221N 00640E,0,NLALM
78682,0,NL,AER,Almere,Almere,FL,1-3--6--,AF,2007.0,0,5222N 00513E,0,NLAER
78685,0,NL,APN,Alphen aan den Rijn,Alphen aan den Rijn,ZH,123-----,AF,1607.0,0,5208N 00440E,0,NLAPN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80364,0,NL,WBR,Westerbroek,Westerbroek,GR,--3-----,RL,701.0,0,5311N 00641E,0,NLWBR
80378,0,NL,WTE,West-Terschelling,West-Terschelling,0,1-------,AF,9602.0,0,0,0,NLWTE
80438,0,NL,WMO,Wormer,Wormer,0,1-3-----,AF,1101.0,0,5230N 00449E,0,NLWMO
80450,0,NL,ZAA,Zaandam,Zaandam,0,123-----,AF,9602.0,0,0,0,NLZAA


In [52]:
df_benelux.fillna(0, inplace=True)

In [109]:
for key, coords in enumerate(df_h_nl.coords):
    lat_lon=[]
    if coords!=0:
        for j in range(2):
            part_a = coords.split()[j][:2]
            part_b = coords.split()[j][-3:-1]
            cor = (str(part_a)+"°"+str(part_b))
            lat_lon.append(cor)
        print(lat_lon)
        df_h_nl.coords[key] = tuple(lat_lon)

['52°38', '00°45']
['52°21', '00°40']
['52°22', '00°13']
['52°08', '00°40']
['52°24', '00°49']
['51°30', '00°18']
['51°02', '00°49']


C:\Users\Fabia\AppData\Local\Temp\ipykernel_26176\271693257.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h_nl.coords[key] = tuple(lat_lon)


['51°23', '00°34']
['51°44', '00°53']
['51°53', '00°40']
['52°31', '00°56']
['52°57', '00°46']
['51°47', '00°38']
['53°11', '00°26']
['52°28', '00°36']
['52°28', '00°48']
['51°41', '00°34']
['51°58', '00°37']
['51°39', '00°53']
['51°46', '00°32']
['51°53', '00°23']
['51°55', '00°30']
['51°42', '00°19']
['52°34', '00°38']
['51°28', '00°49']
['52°05', '00°08']
['52°28', '00°39']
['51°22', '00°10']
['51°27', '00°42']
['51°32', '00°05']
['53°11', '00°41']
['52°30', '00°49']


In [102]:
df_h_nl

,mutation,country,city_abbr,city_full,city_full_2,subdivision,function,status,date,iata_code,coords,comments,harbour_code,lat,lon
0,0,NL,ABL,Alblasserdam,Alblasserdam,0,1-------,AF,9602.0,0,0,0,NLABL,0,0
1,0,NL,ALK,Alkmaar,Alkmaar,NH,123-----,AF,1601.0,0,"(52°38, 00°45)",0,NLALK,0,0
2,0,NL,ALM,Almelo,Almelo,OV,123-----,AF,1601.0,QYL,"(52°21, 00°40)",0,NLALM,0,0
3,0,NL,AER,Almere,Almere,FL,1-3--6--,AF,2007.0,0,"(52°22, 00°13)",0,NLAER,0,0
4,0,NL,APN,Alphen aan den Rijn,Alphen aan den Rijn,ZH,123-----,AF,1607.0,0,"(52°08, 00°40)",0,NLAPN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0,NL,WBR,Westerbroek,Westerbroek,GR,--3-----,RL,701.0,0,"(53°11, 00°41)",0,NLWBR,0,0
65,0,NL,WTE,West-Terschelling,West-Terschelling,0,1-------,AF,9602.0,0,0,0,NLWTE,0,0
66,0,NL,WMO,Wormer,Wormer,0,1-3-----,AF,1101.0,0,"(52°30, 00°49)",0,NLWMO,0,0
67,0,NL,ZAA,Zaandam,Zaandam,0,123-----,AF,9602.0,0,0,0,NLZAA,0,0
